AutoGluon - Predicción de ventas (tn) por producto para febrero 2020

In [1]:
!pip install autogluon.timeseries

In [2]:
# Importar librerías
import pandas as pd
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame

/home/nespina/Documentos/austral/labo_3/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Cargar datasets
df_sellin = pd.read_csv("../data/sell-in.txt", sep="\t")
df_productos = pd.read_csv("../data/tb_productos.txt", sep="\t")

# Leer lista de productos a predecir
with open("../data/product_id_apredecir201912.txt", "r") as f:
    product_ids = [int(line.strip()) for line in f if line.strip().isdigit()]

# Preprocesamiento
# Convertir periodo a datetime
df_sellin['timestamp'] = pd.to_datetime(df_sellin['periodo'], format='%Y%m')

In [4]:
# Filtrar hasta dic 2019 y productos requeridos
df_filtered = df_sellin[
    (df_sellin['timestamp'] <= '2019-12-01') &
    (df_sellin['product_id'].isin(product_ids))
]

In [5]:
# Agregar tn por periodo, cliente y producto
df_grouped = df_filtered.groupby(['timestamp', 'customer_id', 'product_id'], as_index=False)['tn'].sum()

In [6]:
# Agregar tn total por periodo y producto
df_monthly_product = df_grouped.groupby(['timestamp', 'product_id'], as_index=False)['tn'].sum()

In [7]:
# Agregar columna 'item_id' para AutoGluon
df_monthly_product['item_id'] = df_monthly_product['product_id']

In [8]:
# # ⏰ 4. Crear TimeSeriesDataFrame
# ts_data = TimeSeriesDataFrame.from_data_frame(
#     df_monthly_product,
#     id_column='item_id',
#     timestamp_column='timestamp'
# )
# Completar valores faltantes
# ts_data = ts_data.fill_missing_values()

In [9]:
# --- Inserta este código en una nueva celda después de la celda [9] ---

# Hacemos una copia para mantener el dataframe original intacto
df_with_lags = df_monthly_product.copy()

# Es fundamental ordenar por producto (item_id) y fecha (timestamp)
# para que el cálculo de los lags sea correcto para cada serie individual.
df_with_lags = df_with_lags.sort_values(by=['item_id', 'timestamp'])

# Usamos el método shift() de pandas para crear los lags.
# El groupby('item_id') es CRUCIAL para asegurar que los lags se calculan
# dentro de cada serie de producto y no se mezclen datos entre productos.
print("Creando lags de 12 meses...")
for i in range(1, 13):
    df_with_lags[f'tn_lag_{i}'] = df_with_lags.groupby('item_id')['tn'].shift(i)

# Nota: El método shift() introducirá valores NaN al principio de cada serie
# (ej. los primeros 3 meses para el lag 3). Esto es normal.
# AutoGluon puede manejar estos NaNs en las covariables.

print("\nAsí se ven las primeras filas del DataFrame con los nuevos lags:")
# Mostramos un producto específico para ver los lags en acción
print(df_with_lags[df_with_lags['item_id'] == 20001].head())


# --- Ahora, modifica tu celda [11] para usar este nuevo DataFrame ---

# 4. Crear TimeSeriesDataFrame (usando el dataframe con lags)
# AutoGluon detectará automáticamente las columnas 'tn_lag_...' como 'past_covariates'
ts_data = TimeSeriesDataFrame.from_data_frame(
    df_with_lags,       # <--- ¡Asegúrate de usar el nuevo DataFrame!
    id_column='item_id',
    timestamp_column='timestamp'
)

ts_data = ts_data.fill_missing_values()


Creando lags de 12 meses...

Así se ven las primeras filas del DataFrame con los nuevos lags:
      timestamp  product_id          tn  item_id    tn_lag_1    tn_lag_2  \
0    2017-01-01       20001   934.77222    20001         NaN         NaN   
496  2017-02-01       20001   798.01620    20001   934.77222         NaN   
994  2017-03-01       20001  1303.35771    20001   798.01620   934.77222   
1495 2017-04-01       20001  1069.96130    20001  1303.35771   798.01620   
1995 2017-05-01       20001  1502.20132    20001  1069.96130  1303.35771   

       tn_lag_3   tn_lag_4  tn_lag_5  tn_lag_6  tn_lag_7  tn_lag_8  tn_lag_9  \
0           NaN        NaN       NaN       NaN       NaN       NaN       NaN   
496         NaN        NaN       NaN       NaN       NaN       NaN       NaN   
994         NaN        NaN       NaN       NaN       NaN       NaN       NaN   
1495  934.77222        NaN       NaN       NaN       NaN       NaN       NaN   
1995  798.01620  934.77222       NaN       NaN   

In [ ]:
# ⚙️ 5. Definir y entrenar predictor
predictor = TimeSeriesPredictor(
    prediction_length=2,
    target='tn',
    freq='MS'  # Frecuencia mensual (Month Start), 
)

predictor.fit(ts_data, num_val_windows=2, time_limit=60*60*4)

Beginning AutoGluon training... Time limit = 14400s
AutoGluon will save models to '/home/nespina/Documentos/austral/labo_3/src/AutogluonModels/ag-20250719_024324'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.12.3
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #66-Ubuntu SMP PREEMPT_DYNAMIC Fri Jun 13 20:25:30 UTC 2025
CPU Count:          8
GPU Count:          0
Memory Avail:       5.64 GB / 11.37 GB (49.6%)
Disk Space Avail:   107.98 GB / 284.85 GB (37.9%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'MS',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'tn',
 'time_limit': 14400,
 'verbosity': 2}

train_data with frequency 'IRREG'

In [ ]:
# 🔮 6. Generar predicción
forecast = predictor.predict(ts_data)

In [ ]:
# Extraer predicción media y filtrar febrero 2020
forecast_mean = forecast['mean'].reset_index()
print(forecast_mean.columns)

In [ ]:
# Tomar solo item_id y la predicción 'mean'
resultado = forecast['mean'].reset_index()[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']

# Filtrar solo febrero 2020
resultado = forecast['mean'].reset_index()
resultado = resultado[resultado['timestamp'] == '2020-02-01']

# Renombrar columnas
resultado = resultado[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']


In [ ]:
# 💾 7. Guardar archivo
resultado.to_csv("predicciones_febrero2020_fecha_01_07-autogluon-lags.csv", index=False)
resultado.head()

In [ ]:
# Mostrar los mejores modelos del predictor
print("Mejores modelos entrenados:")
print(predictor.leaderboard())